In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split,GridSearchCV, KFold
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_auc_score

In [90]:
data = pd.read_csv("D:/propycharm2023/mq5/KDATA_type.csv")
data = data.drop(['Unnamed: 0', '时间', '订单规定价格', '订单ID', 'profit', '类型'], axis=1, inplace=False)
data = data.dropna(axis=0)
data.describe()

,开盘价,最高价,最低价,收盘价,成交量,atr5,atr10,atr14,macd,sma5,...,cci,roc2,roc3,roc5,slowk,slowd,upper_band,middle_band,lower_band,type
count,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,...,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.000000,4837.00000,4837.000000
mean,1266.551890,1275.789010,1256.886322,1266.604966,73573.655778,19.038853,19.032847,19.028718,2.183051,1265.978518,...,7.888379,9.999791,17.049289,14.927825,53.540808,53.537682,1285.643723,1265.978518,1246.31326,0.472400
std,420.863085,423.254268,418.099374,420.747296,58801.914960,9.871073,9.249716,9.012476,14.325182,420.693732,...,91.325223,62.949605,84.556620,77.239603,25.881355,24.193892,426.051455,420.693732,415.78655,0.499289
min,412.200000,413.300000,409.800000,412.300000,2.000000,2.740000,3.230000,3.290000,-55.680000,413.820000,...,-166.670000,0.000000,-4.810000,0.000000,2.710000,3.210000,417.050000,413.820000,407.87000,0.000000
25%,972.400000,983.700000,959.300000,972.700000,22874.000000,12.640000,12.760000,12.870000,-5.870000,968.380000,...,-70.580000,0.000000,0.000000,0.000000,30.640000,32.550000,992.050000,968.380000,946.80000,0.000000
50%,1277.490000,1284.070000,1270.880000,1277.180000,68817.000000,17.450000,17.640000,17.830000,1.650000,1278.200000,...,17.290000,0.000000,0.000000,0.000000,55.560000,55.410000,1292.630000,1278.200000,1262.50000,0.000000
75%,1619.700000,1632.350000,1608.130000,1620.130000,110068.000000,23.530000,23.590000,23.750000,10.790000,1618.780000,...,84.430000,0.000000,0.000000,0.000000,76.740000,75.060000,1652.870000,1618.780000,1593.92000,1.000000
max,2066.550000,2074.870000,2034.440000,2063.250000,679039.000000,91.220000,74.260000,66.110000,69.050000,2036.300000,...,166.670000,475.100000,477.280000,473.760000,98.150000,96.010000,2120.490000,2036.300000,2007.94000,1.000000


In [91]:
#特征和标签
X = data.drop(columns = ['type'])
Y = data['type']
#特征和标签
X = data.drop(columns = ['type'])
Y = data['type']

#标准化StandardScaler()
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train,X_test,Y_train,Y_test = X[:-500],X[-500:],Y[:-500],Y[-500:]

In [83]:
# 创建数据集对象
# train_pool = Pool(X_train, Y_train)

# 定义参数网格 
params = {
    'iterations': [1000, 1500],
    'learning_rate': [0.02],
    'depth': [6, 8],
    'rsm':[0.8,0.9],
    'fold_len_multiplier':[1.1,1.2,1.3]
}

# 设置其他固定参数
fixed_params = {
    'early_stopping_rounds': 10,
    'loss_function':"Logloss",
    'od_type':"Iter",
}

# 执行网格搜索
model = CatBoostClassifier(**fixed_params)
grid_search   = GridSearchCV(model, param_grid = params, scoring="roc_auc", cv = 5)

grid_search .fit(X_train[:-500],Y_train[:-500],eval_set=(X_train[-500:],Y_train[-500:]))

# 输出最佳得分和最佳参数组合
print("Best Score:", grid_search.best_score_)
print("Best Params:", grid_search.best_params_)


0:	learn: 0.6919423	test: 0.6932771	best: 0.6932771 (0)	total: 5.07ms	remaining: 5.07s
1:	learn: 0.6909583	test: 0.6933698	best: 0.6932771 (0)	total: 10.3ms	remaining: 5.13s
2:	learn: 0.6900223	test: 0.6933802	best: 0.6932771 (0)	total: 14.3ms	remaining: 4.76s
3:	learn: 0.6889371	test: 0.6934334	best: 0.6932771 (0)	total: 18.5ms	remaining: 4.62s
4:	learn: 0.6881880	test: 0.6934116	best: 0.6932771 (0)	total: 21.6ms	remaining: 4.31s
5:	learn: 0.6874886	test: 0.6934175	best: 0.6932771 (0)	total: 25.4ms	remaining: 4.21s
6:	learn: 0.6867296	test: 0.6934640	best: 0.6932771 (0)	total: 28.6ms	remaining: 4.06s
7:	learn: 0.6860361	test: 0.6936710	best: 0.6932771 (0)	total: 31.8ms	remaining: 3.94s
8:	learn: 0.6853977	test: 0.6937485	best: 0.6932771 (0)	total: 35.3ms	remaining: 3.89s
9:	learn: 0.6844167	test: 0.6937996	best: 0.6932771 (0)	total: 38.7ms	remaining: 3.83s
10:	learn: 0.6840018	test: 0.6938401	best: 0.6932771 (0)	total: 41.6ms	remaining: 3.74s
Stopped by overfitting detector  (10 itera

12:	learn: 0.6830555	test: 0.6931962	best: 0.6927694 (4)	total: 30.3ms	remaining: 2.3s
13:	learn: 0.6824276	test: 0.6932704	best: 0.6927694 (4)	total: 33.2ms	remaining: 2.34s
14:	learn: 0.6816712	test: 0.6932991	best: 0.6927694 (4)	total: 35.7ms	remaining: 2.35s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.6927694459
bestIteration = 4

Shrink model to first 5 iterations.
0:	learn: 0.6922927	test: 0.6931129	best: 0.6931129 (0)	total: 2.06ms	remaining: 2.05s
1:	learn: 0.6918451	test: 0.6930774	best: 0.6930774 (1)	total: 4.35ms	remaining: 2.17s
2:	learn: 0.6904096	test: 0.6930353	best: 0.6930353 (2)	total: 6.62ms	remaining: 2.2s
3:	learn: 0.6901388	test: 0.6929851	best: 0.6929851 (3)	total: 9.01ms	remaining: 2.24s
4:	learn: 0.6892281	test: 0.6930168	best: 0.6929851 (3)	total: 11.3ms	remaining: 2.25s
5:	learn: 0.6881121	test: 0.6929445	best: 0.6929445 (5)	total: 13.7ms	remaining: 2.27s
6:	learn: 0.6877355	test: 0.6929236	best: 0.6929236 (6)	total: 16.1ms	remaining: 2

36:	learn: 0.6726594	test: 0.6918846	best: 0.6917032 (35)	total: 85.4ms	remaining: 3.38s
37:	learn: 0.6722864	test: 0.6922464	best: 0.6917032 (35)	total: 88.2ms	remaining: 3.39s
38:	learn: 0.6718154	test: 0.6922389	best: 0.6917032 (35)	total: 90.4ms	remaining: 3.39s
39:	learn: 0.6714553	test: 0.6921834	best: 0.6917032 (35)	total: 92.8ms	remaining: 3.38s
40:	learn: 0.6709567	test: 0.6922138	best: 0.6917032 (35)	total: 95.3ms	remaining: 3.39s
41:	learn: 0.6704763	test: 0.6921700	best: 0.6917032 (35)	total: 97.9ms	remaining: 3.4s
42:	learn: 0.6702140	test: 0.6924641	best: 0.6917032 (35)	total: 100ms	remaining: 3.4s
43:	learn: 0.6699068	test: 0.6924390	best: 0.6917032 (35)	total: 103ms	remaining: 3.4s
44:	learn: 0.6694777	test: 0.6925989	best: 0.6917032 (35)	total: 105ms	remaining: 3.39s
45:	learn: 0.6689777	test: 0.6926842	best: 0.6917032 (35)	total: 107ms	remaining: 3.4s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.6917031838
bestIteration = 35

Shrink model to fir

3:	learn: 0.6907912	test: 0.6933593	best: 0.6931007 (0)	total: 10ms	remaining: 3.75s
4:	learn: 0.6901972	test: 0.6934151	best: 0.6931007 (0)	total: 12.5ms	remaining: 3.74s
5:	learn: 0.6897122	test: 0.6934466	best: 0.6931007 (0)	total: 15ms	remaining: 3.73s
6:	learn: 0.6891613	test: 0.6935859	best: 0.6931007 (0)	total: 17.4ms	remaining: 3.7s
7:	learn: 0.6886424	test: 0.6937756	best: 0.6931007 (0)	total: 19.6ms	remaining: 3.66s
8:	learn: 0.6881051	test: 0.6939322	best: 0.6931007 (0)	total: 22ms	remaining: 3.64s
9:	learn: 0.6876922	test: 0.6940089	best: 0.6931007 (0)	total: 24.7ms	remaining: 3.67s
10:	learn: 0.6870834	test: 0.6939983	best: 0.6931007 (0)	total: 27.1ms	remaining: 3.67s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.6931006819
bestIteration = 0

Shrink model to first 1 iterations.
0:	learn: 0.6927300	test: 0.6931258	best: 0.6931258 (0)	total: 2.16ms	remaining: 3.24s
1:	learn: 0.6920683	test: 0.6931758	best: 0.6931258 (0)	total: 4.51ms	remaining: 3.38s
2:

0:	learn: 0.6928086	test: 0.6931287	best: 0.6931287 (0)	total: 2.56ms	remaining: 2.56s
1:	learn: 0.6923986	test: 0.6931990	best: 0.6931287 (0)	total: 5.26ms	remaining: 2.62s
2:	learn: 0.6919429	test: 0.6933246	best: 0.6931287 (0)	total: 7.79ms	remaining: 2.59s
3:	learn: 0.6915559	test: 0.6933415	best: 0.6931287 (0)	total: 10.3ms	remaining: 2.58s
4:	learn: 0.6910187	test: 0.6933786	best: 0.6931287 (0)	total: 12.9ms	remaining: 2.57s
5:	learn: 0.6907079	test: 0.6934284	best: 0.6931287 (0)	total: 15.4ms	remaining: 2.55s
6:	learn: 0.6902522	test: 0.6935014	best: 0.6931287 (0)	total: 17.9ms	remaining: 2.53s
7:	learn: 0.6899368	test: 0.6935728	best: 0.6931287 (0)	total: 20.5ms	remaining: 2.54s
8:	learn: 0.6895676	test: 0.6936893	best: 0.6931287 (0)	total: 23.3ms	remaining: 2.57s
9:	learn: 0.6892249	test: 0.6937720	best: 0.6931287 (0)	total: 25.8ms	remaining: 2.55s
10:	learn: 0.6886153	test: 0.6936904	best: 0.6931287 (0)	total: 28.3ms	remaining: 2.55s
Stopped by overfitting detector  (10 itera

...

49:	learn: 0.6542913	test: 0.6902318	best: 0.6902318 (49)	total: 327ms	remaining: 9.49s
50:	learn: 0.6538138	test: 0.6902284	best: 0.6902284 (50)	total: 334ms	remaining: 9.48s
51:	learn: 0.6532628	test: 0.6902591	best: 0.6902284 (50)	total: 340ms	remaining: 9.46s
52:	learn: 0.6527666	test: 0.6902776	best: 0.6902284 (50)	total: 345ms	remaining: 9.42s
53:	learn: 0.6522767	test: 0.6901385	best: 0.6901385 (53)	total: 351ms	remaining: 9.4s
54:	learn: 0.6515824	test: 0.6902365	best: 0.6901385 (53)	total: 357ms	remaining: 9.38s
55:	learn: 0.6509379	test: 0.6907088	best: 0.6901385 (53)	total: 363ms	remaining: 9.36s
56:	learn: 0.6505348	test: 0.6909036	best: 0.6901385 (53)	total: 369ms	remaining: 9.33s
57:	learn: 0.6502066	test: 0.6910891	best: 0.6901385 (53)	total: 374ms	remaining: 9.3s
58:	learn: 0.6497489	test: 0.6910215	best: 0.6901385 (53)	total: 380ms	remaining: 9.29s
59:	learn: 0.6493306	test: 0.6910785	best: 0.6901385 (53)	total: 386ms	remaining: 9.27s
60:	learn: 0.6491822	test: 0.69109

0:	learn: 0.6922855	test: 0.6927625	best: 0.6927625 (0)	total: 5.37ms	remaining: 8.04s
1:	learn: 0.6917652	test: 0.6927662	best: 0.6927625 (0)	total: 11.7ms	remaining: 8.73s
2:	learn: 0.6906474	test: 0.6927967	best: 0.6927625 (0)	total: 17.9ms	remaining: 8.96s
3:	learn: 0.6900470	test: 0.6928875	best: 0.6927625 (0)	total: 24.2ms	remaining: 9.06s
4:	learn: 0.6893901	test: 0.6929242	best: 0.6927625 (0)	total: 30.5ms	remaining: 9.12s
5:	learn: 0.6888808	test: 0.6929769	best: 0.6927625 (0)	total: 36.9ms	remaining: 9.18s
6:	learn: 0.6881512	test: 0.6929350	best: 0.6927625 (0)	total: 43.5ms	remaining: 9.27s
7:	learn: 0.6866384	test: 0.6930288	best: 0.6927625 (0)	total: 49.7ms	remaining: 9.28s
8:	learn: 0.6860082	test: 0.6931273	best: 0.6927625 (0)	total: 55.8ms	remaining: 9.24s
9:	learn: 0.6850181	test: 0.6932419	best: 0.6927625 (0)	total: 62.8ms	remaining: 9.35s
10:	learn: 0.6844636	test: 0.6934391	best: 0.6927625 (0)	total: 68.9ms	remaining: 9.32s
Stopped by overfitting detector  (10 itera

2:	learn: 0.6904757	test: 0.6928627	best: 0.6928627 (2)	total: 18ms	remaining: 5.98s
3:	learn: 0.6891383	test: 0.6930411	best: 0.6928627 (2)	total: 24.7ms	remaining: 6.14s
4:	learn: 0.6881853	test: 0.6932628	best: 0.6928627 (2)	total: 33.4ms	remaining: 6.65s
5:	learn: 0.6867869	test: 0.6932862	best: 0.6928627 (2)	total: 41.2ms	remaining: 6.82s
6:	learn: 0.6860666	test: 0.6932103	best: 0.6928627 (2)	total: 50ms	remaining: 7.09s
7:	learn: 0.6850369	test: 0.6933772	best: 0.6928627 (2)	total: 58.1ms	remaining: 7.21s
8:	learn: 0.6843054	test: 0.6933487	best: 0.6928627 (2)	total: 65.8ms	remaining: 7.25s
9:	learn: 0.6833227	test: 0.6935479	best: 0.6928627 (2)	total: 74.3ms	remaining: 7.35s
10:	learn: 0.6824910	test: 0.6933307	best: 0.6928627 (2)	total: 82.2ms	remaining: 7.39s
11:	learn: 0.6821056	test: 0.6935193	best: 0.6928627 (2)	total: 88.7ms	remaining: 7.31s
12:	learn: 0.6812967	test: 0.6936633	best: 0.6928627 (2)	total: 98.3ms	remaining: 7.46s
Stopped by overfitting detector  (10 iterati

2:	learn: 0.6891221	test: 0.6929296	best: 0.6929296 (2)	total: 16.9ms	remaining: 8.42s
3:	learn: 0.6877977	test: 0.6927291	best: 0.6927291 (3)	total: 23.6ms	remaining: 8.83s
4:	learn: 0.6870584	test: 0.6926760	best: 0.6926760 (4)	total: 30.1ms	remaining: 8.99s
5:	learn: 0.6858673	test: 0.6926426	best: 0.6926426 (5)	total: 36.6ms	remaining: 9.12s
6:	learn: 0.6845636	test: 0.6925672	best: 0.6925672 (6)	total: 43.3ms	remaining: 9.22s
7:	learn: 0.6836928	test: 0.6924849	best: 0.6924849 (7)	total: 49.8ms	remaining: 9.3s
8:	learn: 0.6820844	test: 0.6924143	best: 0.6924143 (8)	total: 56ms	remaining: 9.28s
9:	learn: 0.6812145	test: 0.6923436	best: 0.6923436 (9)	total: 62.8ms	remaining: 9.36s
10:	learn: 0.6804118	test: 0.6923877	best: 0.6923436 (9)	total: 69.8ms	remaining: 9.45s
11:	learn: 0.6797466	test: 0.6924822	best: 0.6923436 (9)	total: 75.3ms	remaining: 9.34s
12:	learn: 0.6788123	test: 0.6923035	best: 0.6923035 (12)	total: 82.9ms	remaining: 9.49s
13:	learn: 0.6783258	test: 0.6923389	best:

0:	learn: 0.6918443	test: 0.6931515	best: 0.6931515 (0)	total: 6.5ms	remaining: 9.74s
1:	learn: 0.6902837	test: 0.6931115	best: 0.6931115 (1)	total: 13.7ms	remaining: 10.3s
2:	learn: 0.6892996	test: 0.6929997	best: 0.6929997 (2)	total: 20.6ms	remaining: 10.3s
3:	learn: 0.6883345	test: 0.6930223	best: 0.6929997 (2)	total: 27.3ms	remaining: 10.2s
4:	learn: 0.6866864	test: 0.6928604	best: 0.6928604 (4)	total: 34.1ms	remaining: 10.2s
5:	learn: 0.6858192	test: 0.6928883	best: 0.6928604 (4)	total: 40.7ms	remaining: 10.1s
6:	learn: 0.6847007	test: 0.6928684	best: 0.6928604 (4)	total: 47.5ms	remaining: 10.1s
7:	learn: 0.6834549	test: 0.6934774	best: 0.6928604 (4)	total: 54.1ms	remaining: 10.1s
8:	learn: 0.6828294	test: 0.6933617	best: 0.6928604 (4)	total: 60.5ms	remaining: 10s
9:	learn: 0.6816529	test: 0.6933124	best: 0.6928604 (4)	total: 67ms	remaining: 9.99s
10:	learn: 0.6811764	test: 0.6933778	best: 0.6928604 (4)	total: 73.2ms	remaining: 9.91s
11:	learn: 0.6803673	test: 0.6934433	best: 0.69

In [97]:
# train_pool = Pool(X_train, Y_train)
params = {
    'iterations': 1000,
    'learning_rate': 0.02,
    'depth': 8,
    'early_stopping_rounds': 10,
    'loss_function':"Logloss",
    
}
# 'od_type':"Iter"
model = CatBoostClassifier(**params)
model.fit(X_train,Y_train)
#cat_features 具有分类特征的数据
# col=['grade','subGrade','employmentTitle','homeOwnership','verificationStatus']
# clf = model.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=500,cat_features=col)
# prediction_type='Probability' 输出概率
# yy_pred_valid=clf.predict(x_test,prediction_type='Probability')


0:	learn: 0.6926042	total: 13.6ms	remaining: 13.6s
1:	learn: 0.6919048	total: 23.3ms	remaining: 11.6s
2:	learn: 0.6914012	total: 33.3ms	remaining: 11.1s
3:	learn: 0.6908883	total: 42.8ms	remaining: 10.7s
4:	learn: 0.6902527	total: 49.1ms	remaining: 9.78s
5:	learn: 0.6897452	total: 55.2ms	remaining: 9.15s
6:	learn: 0.6894506	total: 60.7ms	remaining: 8.61s
7:	learn: 0.6890125	total: 66.3ms	remaining: 8.21s
8:	learn: 0.6879112	total: 71.8ms	remaining: 7.9s
9:	learn: 0.6872061	total: 76.1ms	remaining: 7.53s
10:	learn: 0.6867545	total: 80.3ms	remaining: 7.22s
11:	learn: 0.6863414	total: 84.6ms	remaining: 6.96s
12:	learn: 0.6857133	total: 88.8ms	remaining: 6.74s
13:	learn: 0.6850166	total: 93.1ms	remaining: 6.56s
14:	learn: 0.6847054	total: 97.3ms	remaining: 6.39s
15:	learn: 0.6842726	total: 102ms	remaining: 6.25s
16:	learn: 0.6839646	total: 106ms	remaining: 6.13s
17:	learn: 0.6836039	total: 110ms	remaining: 6.01s
18:	learn: 0.6830885	total: 114ms	remaining: 5.9s
19:	learn: 0.6825888	total: 

181:	learn: 0.6267646	total: 830ms	remaining: 3.73s
182:	learn: 0.6265774	total: 835ms	remaining: 3.73s
183:	learn: 0.6261041	total: 840ms	remaining: 3.72s
184:	learn: 0.6257645	total: 845ms	remaining: 3.72s
185:	learn: 0.6253783	total: 849ms	remaining: 3.71s
186:	learn: 0.6252866	total: 854ms	remaining: 3.71s
187:	learn: 0.6250386	total: 858ms	remaining: 3.71s
188:	learn: 0.6247706	total: 862ms	remaining: 3.7s
189:	learn: 0.6244782	total: 867ms	remaining: 3.69s
190:	learn: 0.6241725	total: 871ms	remaining: 3.69s
191:	learn: 0.6238053	total: 876ms	remaining: 3.68s
192:	learn: 0.6236418	total: 880ms	remaining: 3.68s
193:	learn: 0.6233783	total: 884ms	remaining: 3.67s
194:	learn: 0.6230963	total: 889ms	remaining: 3.67s
195:	learn: 0.6228250	total: 893ms	remaining: 3.66s
196:	learn: 0.6226115	total: 897ms	remaining: 3.66s
197:	learn: 0.6222818	total: 902ms	remaining: 3.65s
198:	learn: 0.6222224	total: 906ms	remaining: 3.65s
199:	learn: 0.6219242	total: 910ms	remaining: 3.64s
200:	learn: 0

360:	learn: 0.5774366	total: 1.65s	remaining: 2.93s
361:	learn: 0.5772189	total: 1.66s	remaining: 2.92s
362:	learn: 0.5770715	total: 1.66s	remaining: 2.92s
363:	learn: 0.5768349	total: 1.67s	remaining: 2.91s
364:	learn: 0.5766492	total: 1.67s	remaining: 2.91s
365:	learn: 0.5762692	total: 1.68s	remaining: 2.9s
366:	learn: 0.5760476	total: 1.68s	remaining: 2.9s
367:	learn: 0.5758003	total: 1.69s	remaining: 2.9s
368:	learn: 0.5755755	total: 1.69s	remaining: 2.89s
369:	learn: 0.5753736	total: 1.69s	remaining: 2.88s
370:	learn: 0.5750348	total: 1.7s	remaining: 2.88s
371:	learn: 0.5748079	total: 1.7s	remaining: 2.88s
372:	learn: 0.5745737	total: 1.71s	remaining: 2.87s
373:	learn: 0.5743035	total: 1.71s	remaining: 2.87s
374:	learn: 0.5739416	total: 1.72s	remaining: 2.86s
375:	learn: 0.5737620	total: 1.72s	remaining: 2.85s
376:	learn: 0.5735489	total: 1.72s	remaining: 2.85s
377:	learn: 0.5730858	total: 1.73s	remaining: 2.85s
378:	learn: 0.5728019	total: 1.73s	remaining: 2.84s
379:	learn: 0.572

535:	learn: 0.5238062	total: 2.46s	remaining: 2.13s
536:	learn: 0.5234003	total: 2.46s	remaining: 2.12s
537:	learn: 0.5231586	total: 2.46s	remaining: 2.12s
538:	learn: 0.5229127	total: 2.47s	remaining: 2.11s
539:	learn: 0.5225650	total: 2.48s	remaining: 2.11s
540:	learn: 0.5223249	total: 2.48s	remaining: 2.1s
541:	learn: 0.5221544	total: 2.48s	remaining: 2.1s
542:	learn: 0.5216608	total: 2.49s	remaining: 2.09s
543:	learn: 0.5215027	total: 2.49s	remaining: 2.09s
544:	learn: 0.5211717	total: 2.5s	remaining: 2.08s
545:	learn: 0.5209676	total: 2.5s	remaining: 2.08s
546:	learn: 0.5205108	total: 2.5s	remaining: 2.08s
547:	learn: 0.5201621	total: 2.51s	remaining: 2.07s
548:	learn: 0.5195890	total: 2.51s	remaining: 2.06s
549:	learn: 0.5192571	total: 2.52s	remaining: 2.06s
550:	learn: 0.5188551	total: 2.52s	remaining: 2.06s
551:	learn: 0.5186080	total: 2.53s	remaining: 2.05s
552:	learn: 0.5182885	total: 2.53s	remaining: 2.05s
553:	learn: 0.5178319	total: 2.54s	remaining: 2.04s
554:	learn: 0.517

715:	learn: 0.4671170	total: 3.26s	remaining: 1.29s
716:	learn: 0.4668431	total: 3.27s	remaining: 1.29s
717:	learn: 0.4667146	total: 3.27s	remaining: 1.28s
718:	learn: 0.4664310	total: 3.27s	remaining: 1.28s
719:	learn: 0.4662023	total: 3.28s	remaining: 1.27s
720:	learn: 0.4655610	total: 3.28s	remaining: 1.27s
721:	learn: 0.4653339	total: 3.29s	remaining: 1.27s
722:	learn: 0.4651051	total: 3.29s	remaining: 1.26s
723:	learn: 0.4648234	total: 3.3s	remaining: 1.26s
724:	learn: 0.4644333	total: 3.3s	remaining: 1.25s
725:	learn: 0.4641082	total: 3.31s	remaining: 1.25s
726:	learn: 0.4639283	total: 3.31s	remaining: 1.24s
727:	learn: 0.4633567	total: 3.32s	remaining: 1.24s
728:	learn: 0.4630193	total: 3.32s	remaining: 1.23s
729:	learn: 0.4628447	total: 3.33s	remaining: 1.23s
730:	learn: 0.4623446	total: 3.33s	remaining: 1.23s
731:	learn: 0.4621113	total: 3.33s	remaining: 1.22s
732:	learn: 0.4619058	total: 3.34s	remaining: 1.22s
733:	learn: 0.4616351	total: 3.34s	remaining: 1.21s
734:	learn: 0.

895:	learn: 0.4203398	total: 4.07s	remaining: 472ms
896:	learn: 0.4201888	total: 4.07s	remaining: 467ms
897:	learn: 0.4200769	total: 4.08s	remaining: 463ms
898:	learn: 0.4199454	total: 4.08s	remaining: 458ms
899:	learn: 0.4198190	total: 4.08s	remaining: 454ms
900:	learn: 0.4195804	total: 4.09s	remaining: 449ms
901:	learn: 0.4193656	total: 4.09s	remaining: 445ms
902:	learn: 0.4192411	total: 4.1s	remaining: 440ms
903:	learn: 0.4189203	total: 4.1s	remaining: 436ms
904:	learn: 0.4186651	total: 4.11s	remaining: 431ms
905:	learn: 0.4184351	total: 4.11s	remaining: 427ms
906:	learn: 0.4182160	total: 4.12s	remaining: 422ms
907:	learn: 0.4179002	total: 4.12s	remaining: 417ms
908:	learn: 0.4177616	total: 4.12s	remaining: 413ms
909:	learn: 0.4173907	total: 4.13s	remaining: 408ms
910:	learn: 0.4171741	total: 4.13s	remaining: 404ms
911:	learn: 0.4169868	total: 4.14s	remaining: 399ms
912:	learn: 0.4166671	total: 4.14s	remaining: 395ms
913:	learn: 0.4164894	total: 4.14s	remaining: 390ms
914:	learn: 0.

In [99]:
# test_pool = Pool(X_test)
#test
y_pred = model.predict(X_test)
#混淆矩阵
cmatrix = confusion_matrix(Y_test, y_pred)
cmatrix[::-1, ::-1]

array([[ 87, 161],
       [ 95, 157]], dtype=int64)

In [100]:
#test
roc_auc = roc_auc_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred)
print("roc_auc:",roc_auc,"Accuracy:", accuracy,"f1:", f1,"recall:", recall,"precision:", precision)

roc_auc: 0.48691116231438814 Accuracy: 0.488 f1: 0.4046511627906977 recall: 0.35080645161290325 precision: 0.47802197802197804


In [ ]:
##################
cv_params = {'iterations': [800,1000,1500],
            'learning_rate':[0.01, 0.02],
            'depth':[3,6],
            'rsm':[0.8,0.9],
            'fold_len_multiplier':[1.1]}
# other_params = {
#     'iterations': 1500,
#     'learning_rate':0.02,
#     'depth':3,
#     'rsm':0.8,
#     'fold_len_multiplier':1.1
# }
#指定category类型的列，可以是索引，也可以是列名
#分类特征，可以通过将其索引添加到cat_features参数中来指定它们
# cat_features = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
# optimized_cb.fit(X,Y,cat_features =category_features)
other_params = {}
model_cb = CatBoostClassifier(**other_params)
optimized_cb = GridSearchCV(estimator=model_cb, param_grid=cv_params, scoring='roc_auc', cv=5, verbose=1, n_jobs=2)
optimized_cb.fit(X_train[:-500],Y_train[:-500],eval_set=(X_train[-500:],Y_train[-500:]))
print('参数的最佳取值：{0}'.format(optimized_cb.best_params_))
print('最佳模型得分:{0}'.format(optimized_cb.best_score_))
# print(optimized_cb.cv_results_['mean_test_score'])
# print(optimized_cb.cv_results_['params'])

# KFold

In [75]:
model=CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            learning_rate=0.02,
            iterations=1000,
            random_seed=2023,
            depth=8,
            rsm=0.9,
            fold_len_multiplier=1.1)

# 定义KFold交叉验证
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
answers = []
roc_auc_score_sum = 0
i = 0
for train, test in kfold.split(X_train, Y_train):  
    x_train = X_train.iloc[train]
    y_train = Y_train.iloc[train]
    x_test = X_train.iloc[test]
    y_test = Y_train.iloc[test]
#     print('x_train', x_train.shape)
#     print('y_train', y_train.shape)
    clf = model.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=500)
    
#     yy_pred_valid=clf.predict(x_test,prediction_type='Probability')[:,-1]
    yy_pred_valid=clf.predict(x_test)
    answers.append(yy_pred_valid)
#     print(yy_pred_valid)
    roc_auc_score_ = roc_auc_score(y_test, yy_pred_valid)
    roc_auc_score_sum += roc_auc_score_
    print('roc_auc_score:{}'.format(roc_auc_score_))
    print('f1_score:{}'.format(f1_score(y_test, yy_pred_valid)))
    print('accuracy_score:{}'.format(accuracy_score(y_test, yy_pred_valid)))
    print('recall_score:{}'.format(recall_score(y_test, yy_pred_valid)))
    i = i + 1
    
print('\nmean roc_auc:{}'.format(roc_auc_score_sum/i))

0:	test: 0.4965840	best: 0.4965840 (0)	total: 11.4ms	remaining: 11.4s
500:	test: 0.5337699	best: 0.5345724 (169)	total: 3.37s	remaining: 3.36s
999:	test: 0.5373224	best: 0.5387188 (796)	total: 6.71s	remaining: 0us

bestTest = 0.5387187554
bestIteration = 796

Shrink model to first 797 iterations.
roc_auc_score:0.5230696798493408
f1_score:0.46315789473684205
accuracy_score:0.5299539170506913
recall_score:0.4444444444444444
0:	test: 0.5151653	best: 0.5151653 (0)	total: 6.01ms	remaining: 6.01s
500:	test: 0.5022687	best: 0.5295221 (2)	total: 3.34s	remaining: 3.32s
999:	test: 0.5038698	best: 0.5295221 (2)	total: 6.72s	remaining: 0us

bestTest = 0.5295220618
bestIteration = 2

Shrink model to first 3 iterations.
roc_auc_score:0.5397457379185617
f1_score:0.43776824034334766
accuracy_score:0.5472350230414746
recall_score:0.3686746987951807
0:	test: 0.5034975	best: 0.5034975 (0)	total: 5.37ms	remaining: 5.36s
500:	test: 0.4867873	best: 0.5130610 (3)	total: 3.36s	remaining: 3.34s
999:	test: 0.48

array(0.)